In [9]:
import pandas as pd
import numpy as np
import geopandas as gpd

## Q1: Find the 3-month period with the highest total acres burned since Jan 2000, according to NOAA. What was the total acres burned in that period?

In [10]:
df = pd.read_csv("../data/wildfire/input/noaa_wildfires.csv", skiprows=3)

In [11]:
df = df.sort_values('Date')
    
# Create a function to check if dates are consecutive months
def are_consecutive(date1, date2):
    year1, month1 = divmod(date1, 100)
    year2, month2 = divmod(date2, 100)

    # Convert to total months
    total_months1 = year1 * 12 + month1
    total_months2 = year2 * 12 + month2

    return total_months2 - total_months1 == 1

max_acres = 0
max_period = None

# Iterate through possible 3-month periods
for i in range(len(df) - 2):
    # Check if months are consecutive
    if (are_consecutive(df['Date'].iloc[i], df['Date'].iloc[i+1]) and 
        are_consecutive(df['Date'].iloc[i+1], df['Date'].iloc[i+2])):

        # Calculate total acres for this period
        period_acres = df['Acres Burned'].iloc[i:i+3].sum()

        # Update maximum if this period is larger
        if period_acres > max_acres:
            max_acres = period_acres
            max_period = df.iloc[i:i+3]

print({
    'Start Date': max_period['Date'].iloc[0],
    'End Date': max_period['Date'].iloc[-1],
    'Total Acres Burned': max_acres,
    'Period Data': max_period
})

{'Start Date': np.int64(201506), 'End Date': np.int64(201508), 'Total Acres Burned': np.int64(7805421), 'Period Data':        Date  Acres Burned  Number of Fires  Acres Burned per Fire
185  201506       1868614             6430                 290.61
186  201507       3461087             8186                 422.81
187  201508       2475720             7555                 327.69}


## Q2: Which NIFC geographic area intersects with the most US states? Give the abbreviation of the geographic area.

In [12]:
gdf_usa = gpd.read_file("../data/wildfire/input/usa.gpkg")
gdf_nifc = gpd.read_file("../data/wildfire/input/nifc_geographic_areas.gpkg")

/Users/eylai/Projects/LLMBenchmark/.venv/lib/python3.10/site-packages/pyogrio/raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column DateCurrent, 2022-01-03T16:37:52.0Z, successfully parsed
  return ogr_read(


In [13]:
# Ensure same CRS
gdf_nifc = gdf_nifc.to_crs(gdf_usa.crs)

# Dissolve state geometries
gdf_usa_dissolved = gdf_usa.dissolve(by='adm1_name', as_index=False)

# Perform spatial join
joined = gpd.sjoin(gdf_usa_dissolved, gdf_nifc, how='right', predicate='intersects')

# Group by NIFC region and aggregate both count and list of states
result = (joined.groupby(['GACCName', 'GACCAbbreviation'])
          .agg({
              'adm1_name': [('Number_of_States', 'nunique'),
                           ('States', lambda x: list(set(x)))]
          })
          .reset_index())

# Flatten column names
result.columns = ['GACC_Name', 'GACC_Abbreviation', 'Number_of_States', 'States']

# Sort by number of states (descending)
result = result.sort_values('Number_of_States', ascending=False)

# Get the region(s) with the maximum number of states
max_states = result['Number_of_States'].max()
top_regions = result[result['Number_of_States'] == max_states]

print("\nNIFC Geographic Area(s) intersecting with the most states:")
for _, row in top_regions.iterrows():
    print(f"\nGACC Name: {row['GACC_Name']}")
    print(f"GACC Abbreviation: {row['GACC_Abbreviation']}")
    print(f"Number of intersecting states: {row['Number_of_States']}")
    print("States:", ", ".join(sorted(row['States'])))


NIFC Geographic Area(s) intersecting with the most states:

GACC Name: Eastern Area Coordination Center
GACC Abbreviation: EACC
Number of intersecting states: 30
States: Arkansas, Connecticut, Delaware, District of Columbia, Illinois, Indiana, Iowa, Kansas, Kentucky, Maine, Maryland, Massachusetts, Michigan, Minnesota, Missouri, Nebraska, New Hampshire, New Jersey, New York, North Dakota, Ohio, Oklahoma, Pennsylvania, Rhode Island, South Dakota, Tennessee, Vermont, Virginia, West Virginia, Wisconsin


## Q3: Which US states fall into the most number of NIFC Geographic Areas?

In [14]:
gdf_usa = gpd.read_file("../data/wildfire/input/usa.gpkg")
gdf_nifc = gpd.read_file("../data/wildfire/input/nifc_geographic_areas.gpkg")

/Users/eylai/Projects/LLMBenchmark/.venv/lib/python3.10/site-packages/pyogrio/raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column DateCurrent, 2022-01-03T16:37:52.0Z, successfully parsed
  return ogr_read(


In [15]:
# 1. Ensure both GeoDataFrames have the same CRS (Coordinate Reference System)
gdf_nifc = gdf_nifc.to_crs(gdf_usa.crs)

# 2. Dissolve the state geometries to ensure each state is represented by a single geometry
gdf_usa_dissolved = gdf_usa.dissolve(by='adm1_name', as_index=False)

# 3. Perform a spatial join
joined = gpd.sjoin(gdf_usa_dissolved, gdf_nifc, how='left', predicate='intersects')

# Group by state and aggregate both count and list of NIFC regions
result = (joined.groupby('adm1_name')
          .agg({
              'GACCAbbreviation': [('Number_of_NIFC_Regions', 'nunique'),
                          ('NIFC_Regions', lambda x: list(set(x)))]
          })
          .reset_index())

# Flatten column names
result.columns = ['State', 'Number_of_NIFC_Regions', 'NIFC_Regions']

# Sort by number of regions (descending)
result = result.sort_values('Number_of_NIFC_Regions', ascending=False)

# Display the result
print(result)

                   State  Number_of_NIFC_Regions  \
4             California                       5   
28                Nevada                       5   
5               Colorado                       4   
36              Oklahoma                       4   
37                Oregon                       4   
12                 Idaho                       4   
16                Kansas                       3   
25              Missouri                       3   
23             Minnesota                       3   
34          North Dakota                       3   
31            New Mexico                       3   
26               Montana                       3   
50               Wyoming                       3   
41          South Dakota                       3   
44                  Utah                       3   
13              Illinois                       2   
17              Kentucky                       2   
1                 Alaska                       2   
2           

## Q4: Find the year with the highest suppression cost per acre of human-caused fire. What was the cost per acre, rounded to the nearest cent?

In [16]:
df_costs = pd.read_csv("../data/wildfire/input/nifc_suppression_costs.csv", delimiter='\t')
df_acres = pd.read_csv("../data/wildfire/input/nifc_human_caused_acres.csv", delimiter='\t')

In [17]:
# Clean and convert acres data
df_acres['Total_clean'] = df_acres['Total'].str.replace(',', '').astype(float)

# Clean and convert costs data
df_costs['Total_clean'] = df_costs['Total'].str.replace('$', '').str.replace(',', '').astype(float)

# Merge the two dataframes on Year
merged_df = pd.merge(df_acres, df_costs, on='Year')

# Calculate cost per acre
merged_df['cost_per_acre'] = merged_df['Total_clean_y'] / merged_df['Total_clean_x']

# Find the year with highest cost per acre
max_cost_year = merged_df.loc[merged_df['cost_per_acre'].idxmax()]

# Format the output
print(f"Year with highest suppression cost per acre: {max_cost_year['Year']}")
print(f"Cost per acre: ${max_cost_year['cost_per_acre']:.2f}")
print(f"Total acres burned: {max_cost_year['Total_clean_x']:,.0f}")
print(f"Total suppression cost: ${max_cost_year['Total_clean_y']:,.2f}")

Year with highest suppression cost per acre: 2023
Cost per acre: $2065.10
Total acres burned: 1,533,245
Total suppression cost: $3,166,300,000.00


## Q5: On average, how many more annual fires are reported by NOAA compared to NIFC since 2000? Round to the nearest whole number.

In [18]:
df_noaa = pd.read_csv("../data/wildfire/input/noaa_wildfires.csv", skiprows=3)
df_nifc = pd.read_csv("../data/wildfire/input/nifc_wildfires.csv", delimiter='\t')

In [19]:
# Convert NIFC fires column to numeric, removing commas
df_nifc['Fires_clean'] = df_nifc['Fires'].str.replace(',', '').astype(int)

# Convert NOAA Date to year and group by year, summing the fires
df_noaa['Year'] = df_noaa['Date'].astype(str).str[:4].astype(int)
noaa_annual = df_noaa.groupby('Year')['Number of Fires'].sum().reset_index()

# Merge the two dataframes
merged_df = pd.merge(
    noaa_annual, 
    df_nifc[['Year', 'Fires_clean']], 
    on='Year',
    how='inner'
)

# Calculate difference (NOAA - NIFC) and take mean
avg_difference = (merged_df['Number of Fires'] - merged_df['Fires_clean']).mean()

# Round to nearest whole number
rounded_difference = round(avg_difference)

print(f"On average, NOAA reports {rounded_difference:,} more fires per year than NIFC")

On average, NOAA reports -1,039 more fires per year than NIFC


## Q6: What is the correlation between (1) the difference between the number of NOAA and NIFC-reported fires and (2) the difference between the acres burned by NOAA and NIFC-reported fires, on an annual basis?

In [20]:
df_noaa = pd.read_csv("../data/wildfire/input/noaa_wildfires.csv", skiprows=3)
df_nifc = pd.read_csv("../data/wildfire/input/nifc_wildfires.csv", delimiter='\t')

In [21]:
# Clean NIFC data
df_nifc['Fires_clean'] = df_nifc['Fires'].str.replace(',', '').astype(int)
df_nifc['Acres_clean'] = df_nifc['Acres'].str.replace('*', '', regex=False).str.replace(',', '').astype(int)

# Convert NOAA Date to year and group by year, summing both fires and acres
df_noaa['Year'] = df_noaa['Date'].astype(str).str[:4].astype(int)
noaa_annual = df_noaa.groupby('Year').agg({
    'Number of Fires': 'sum',
    'Acres Burned': 'sum'
}).reset_index()

# Merge the two dataframes
merged_df = pd.merge(
    noaa_annual, 
    df_nifc[['Year', 'Fires_clean', 'Acres_clean']], 
    on='Year',
    how='inner'
)

# Calculate differences
merged_df['fire_difference'] = merged_df['Number of Fires'] - merged_df['Fires_clean']
merged_df['acres_difference'] = merged_df['Acres Burned'] - merged_df['Acres_clean']

# Calculate correlation
correlation = merged_df['fire_difference'].corr(merged_df['acres_difference'])

print(f"Correlation coefficient: {correlation:.3f}")

Correlation coefficient: 0.519


## Q7: Which geographic area had the most anomalous year (by Z-score) in terms of total acres burned compared to its historical annual average, and what year was it in?

In [22]:
df_human = pd.read_csv("../data/wildfire/input/nifc_human_caused_acres.csv", delimiter='\t')
df_lightning = pd.read_csv("../data/wildfire/input/nifc_lightning_caused_acres.csv", delimiter='\t')

In [23]:
# Combine human and lightning caused fires
def process_dataframes(df_human, df_lightning):
    # Clean column names by removing asterisks
    df_human.columns = df_human.columns.str.replace('*', '')
    df_lightning.columns = df_lightning.columns.str.replace('*', '')
    
    # Get geographic columns (exclude Year and Total)
    geo_cols = [col for col in df_human.columns if col not in ['Year', 'Total']]
    
    # Sum the dataframes
    df_total = df_human.copy()
    for col in geo_cols:
        df_total[col] = df_human[col] + df_lightning[col]
    
    # Replace 'N/A' with np.nan and convert to numeric
    for col in geo_cols:
        df_total[col] = pd.to_numeric(df_total[col].replace('N/A', np.nan).str.replace(',', ''))
    
    return df_total, geo_cols

def find_most_anomalous(df_total, geo_cols):
    # Calculate z-score for each geographic area and year
    anomalies = pd.DataFrame()
    anomalies['Year'] = df_total['Year']
    
    for col in geo_cols:
        mean = df_total[col].mean()
        std = df_total[col].std()
        anomalies[col] = (df_total[col] - mean) / std
    
    # Find the most extreme z-score
    # Use abs() to consider both positive and negative anomalies
    max_anomaly = float('-inf')
    max_area = ''
    max_year = None
    max_actual = None
    max_mean = None
    
    for col in geo_cols:
        abs_anomalies = abs(anomalies[col])
        max_idx = abs_anomalies.idxmax()
        if abs_anomalies[max_idx] > max_anomaly:
            max_anomaly = abs_anomalies[max_idx]
            max_area = col
            max_year = df_total.loc[max_idx, 'Year']
            max_actual = df_total.loc[max_idx, col]
            max_mean = df_total[col].mean()
    
    return max_area, max_year, max_actual, max_mean, max_anomaly

df_total, geo_cols = process_dataframes(df_human, df_lightning)

# Find most anomalous case
area, year, actual, mean, z_score = find_most_anomalous(df_total, geo_cols)

print(f"Most anomalous year was {year} in {area}")
print(f"Acres burned: {actual:,.0f}")
print(f"Historical average: {mean:,.0f}")
print(f"Z-score: {z_score:.2f}")
print(f"This was {abs(z_score):.2f} standard deviations from the mean")

Most anomalous year was 2020 in Northern California
Acres burned: 15,490,121,549,012
Historical average: 757,286,558,096
Z-score: 4.68
This was 4.68 standard deviations from the mean
